# Explore and download predictor (input) data used in `fatalities001`

This notebook allows you to query and extract a subset of the predictor data that feed into the VIEWS model each month, aggregated to the VIEWS levels of analysis. 

No other data transforms are applied. 

The queryset specified here matches the predictor data made available via the VIEWS API, where it is updated each month along with the release of our latest conflict predictions. 

**NOTE: Requires a certificate/access to the VIEWS database.**

In [1]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
# sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
# Views 3
from viewser.operations import fetch
from viewser import Queryset, Column
import views_runs
from views_partitioning import data_partitioner, legacy
from stepshift import views
from views_mapper2.label_writer import *

In [2]:
# !viewser tables list

In [3]:
# !viewser tables show 'imfweo_cm'      

In [4]:
# !viewser queryset list

# Specify the output folder

In [5]:
import os
home = os.path.expanduser("~")

#### Option 1: Save to your desktop

In [6]:
desktop = home+'/Desktop/' 

#### Option 2: Save to the VIEWS Dropbox

In [7]:
# Mydropbox = home + '/Dropbox (ViEWS)/ViEWS/' # (Toggle on to save to Dropbox)
# Monthly_updates = Mydropbox + 'DataReleases/MonthlyUpdates/Data/Features/' # (Toggle on to save to Dropbox)

# Fetch data 


In [8]:
StartOfHistory = 121 
EndOfHistory = 512 # Update as needed

## Country-month level (*cm*)

In [9]:
data_cm = (Queryset("fatalities001_API_predictors_cm", "country_month")

# identifiers
             
.with_column(Column('year', from_table = 'month', from_column = 'year_id'))
.with_column(Column('month', from_table = 'month', from_column = 'month'))
.with_column(Column('country_name', from_table = 'country', from_column = 'name'))

# GED

.with_column(Column("ucdp_ged_best_sb", from_table = "ged2_cm", from_column = "ged_sb_best_sum_nokgi"))                   
.with_column(Column("ucdp_ged_best_os", from_table = "ged2_cm", from_column = "ged_os_best_sum_nokgi"))   
.with_column(Column("ucdp_ged_best_ns", from_table = "ged2_cm", from_column = "ged_ns_best_sum_nokgi"))          
  
# ACLED
                    
.with_column(Column("acled_sb_fat", from_table = "acled2_cm", from_column = "acled_sb_fat")) 
.with_column(Column("acled_os_fat", from_table = "acled2_cm", from_column = "acled_os_fat")) 
.with_column(Column("acled_ns_fat", from_table = "acled2_cm", from_column = "acled_ns_fat"))
           
.with_column(Column("acled_prx_fat", from_table = "acled2_cm", from_column = "acled_prx_fat"))
.with_column(Column("acled_bat_gov_fat", from_table = "acled2_cm", from_column = "acled_bat_gov_fat"))
.with_column(Column("acled_bat_reb_fat", from_table = "acled2_cm", from_column = "acled_bat_reb_fat"))
                     
.with_column(Column("acled_sb_count", from_table = "acled2_cm", from_column = "acled_sb_count")) 
.with_column(Column("acled_os_count", from_table = "acled2_cm", from_column = "acled_os_count")) 
.with_column(Column("acled_ns_count", from_table = "acled2_cm", from_column = "acled_ns_count"))
  
.with_column(Column("acled_prx_count", from_table = "acled2_cm", from_column = "acled_prx_count")) 
.with_column(Column("acled_pr_count", from_table = "acled2_cm", from_column = "acled_pr_count")) 
                    
# V-Dem v. 11 
                    
.with_column(Column("vdem_v11_v2x_rule", from_table = "vdem_v11_cy", from_column = "vdem_v2x_rule")) 
.with_column(Column("vdem_v11_v2x_gender", from_table = "vdem_v11_cy", from_column = "vdem_v2x_gender")) 
.with_column(Column("vdem_v11_v2xcl_acjst", from_table = "vdem_v11_cy", from_column = "vdem_v2xcl_acjst")) 
.with_column(Column("vdem_v11_v2xeg_eqdr", from_table = "vdem_v11_cy", from_column = "vdem_v2xeg_eqdr")) 
.with_column(Column("vdem_v11_v2x_libdem", from_table = "vdem_v11_cy", from_column = "vdem_v2x_libdem")) 

# WDI
                 
.with_column(Column("wdi_sp_pop_totl", from_table = "wdi_cy", from_column = "wdi_sp_pop_totl")) 
                    
# IMF WEO

# Please note that I'm still not sure what to use here. I don't know what f_ngdp_rpch and s_ngdp_rpch is – it's not what we're including in the fatalities002 model, but we included f_ngdp_rpch in the API before. Including it along with the rest here for comparison – they do not match at all. What is going on here?
                    
.with_column(Column("imfweo_f_ngdp_rpch", from_table = "imfweo_cy", from_column = "f_ngdp_rpch")) 
.with_column(Column("imfweo_s_ngdp_rpch", from_table = "imfweo_cy", from_column = "s_ngdp_rpch")) 

.with_column(Column("imfweo_ngdp_rpch_tcurrent", from_table = "imfweo_cm", from_column = "ngdp_rpch_tcurrent")) 
.with_column(Column("imfweo_ngdp_rpch_tmin1", from_table = "imfweo_cm", from_column = "ngdp_rpch_tmin1")) 
.with_column(Column("imfweo_ngdp_rpch_tplus1", from_table = "imfweo_cm", from_column = "ngdp_rpch_tplus1")) 
.with_column(Column("imfweo_ngdp_rpch_tplus2", from_table = "imfweo_cm", from_column = "ngdp_rpch_tplus2")) 

                  
# FAOSTAT
   
.with_column(Column("faostat_wheat_price", from_table = "faostat_pp_cm", from_column = "wheat_price")) 
.with_column(Column("faostat_sugar_price", from_table = "faostat_pp_cm", from_column = "sugar_price")) 
.with_column(Column("faostat_milk_price", from_table = "faostat_pp_cm", from_column = "milk_price"))                 
.with_column(Column("faostat_meat_price", from_table = "faostat_pp_cm", from_column = "meat_price")) 
                    
.with_column(Column("faostat_pct_pop_basicdrink", from_table = "faostat_fsec_cy", from_column = "pct_pop_basicdrink")) 
.with_column(Column("faostat_pct_undernourished", from_table = "faostat_fsec_cy", from_column = "pct_undernourished")) 
.with_column(Column("faostat_pcap_fsupply_var", from_table = "faostat_fsec_cy", from_column = "pcap_fsupply_var")) 
        
# FAO AQUASTAT
                 
.with_column(Column("fao_aquastat_renewable_pcapt", from_table = "fao_aqua_cy", from_column = "renewable_pcap")) 
.with_column(Column("fao_aquastat_fresh_withdrawal_pct", from_table = "fao_aqua_cy", from_column = "fresh_withdrawal_pct")) 
.with_column(Column("fao_aquastat_general_efficiency", from_table = "fao_aqua_cy", from_column = "general_efficiency"))
.with_column(Column("fao_aquastat_services_efficiency", from_table = "fao_aqua_cy", from_column = "services_efficiency")) 

              )
      
data_cm = data_cm.publish().fetch()

print(f"A dataset with {len(data_cm.columns)} columns, with "
      f"data between t {min(data_cm.index.get_level_values(0))} "
      f"and {max(data_cm.index.get_level_values(0))}. "
      f"({len(np.unique(data_cm.index.get_level_values(1)))} units)"
     )

 .      o      O  A dataset with 40 columns, with data between t 1 and 852. (213 units)


In [10]:
cm_subset= data_cm.query(f'month_id >= {StartOfHistory} & month_id <= {EndOfHistory}')

cm_subset # displays the subset

year  month         country_name  ucdp_ged_best_sb  \
month_id country_id                                                       
121      1           1990      1               Guyana               0.0   
         2           1990      1             Suriname               0.0   
         3           1990      1  Trinidad and Tobago               0.0   
         4           1990      1            Venezuela               0.0   
         5           1990      1                Samoa               0.0   
...                   ...    ...                  ...               ...   
512      242         2022      8             Tanzania               0.0   
         243         2022      8              Morocco               0.0   
         244         2022      8           Mauritania               0.0   
         245         2022      8                Sudan               7.0   
         246         2022      8          South Sudan               4.0   

                     ucdp_ged_best_os  ucdp_ged_best_ns  acled_sb_fat  \
month_id country_id                                                     
121      1                        0.0               0.0           NaN   
         2                        0.0               0.0           NaN   
         3                        0.0               0.0           NaN   
         4                       12.0               0.0           NaN   
         5                        0.0               0.0           NaN   
...                               ...               ...           ...   
512      242                      0.0               0.0           0.0   
         243                      0.0               0.0           0.0   
         244                      0.0               0.0           0.0   
         245                     23.0               9.0          11.0   
         246                      0.0               0.0           5.0   

                     acled_os_fat  acled_ns_fat  acled_prx_fat  ...  \
month_id country_id                                             ...   
121      1                    NaN           NaN            NaN  ...   
         2                    NaN           NaN            NaN  ...   
         3                    NaN           NaN            NaN  ...   
         4                    NaN           NaN            NaN  ...   
         5                    NaN           NaN            NaN  ...   
...                           ...           ...            ...  ...   
512      242                  0.0           0.0            0.0  ...   
         243                  0.0           0.0            0.0  ...   
         244                  0.0           0.0            1.0  ...   
         245                 31.0          41.0            3.0  ...   
         246                 20.0         108.0            0.0  ...   

                     faostat_sugar_price  faostat_milk_price  \
month_id country_id                                            
121      1                           0.0                 0.0   
         2                           0.0                 0.0   
         3                           0.0                 0.0   
         4                           0.0                 0.0   
         5                           0.0                 0.0   
...                                  ...                 ...   
512      242                         NaN                 NaN   
         243                         NaN                 NaN   
         244                         NaN                 NaN   
         245                         NaN                 NaN   
         246                         NaN                 NaN   

                     faostat_meat_price  faostat_pct_pop_basicdrink  \
month_id country_id                                                   
121      1                          0.0                         NaN   
         2                          0.0                         NaN   
         3                          0.0               

#### Download as .csv

In [11]:
cm_subset.to_csv(desktop+f'API_predictors_cm_{StartOfHistory}-{EndOfHistory}_fatalities001.csv') 

## PRIO-GRID-month (*pgm*) data

In [12]:
data_pgm = (Queryset("fatalities001_API_predictors_pgm", "priogrid_month")

# identifiers
             
.with_column(Column('year', from_table = 'month', from_column = 'year_id'))
.with_column(Column('month', from_table = 'month', from_column = 'month'))
.with_column(Column('country_name', from_table = 'country', from_column = 'name'))

# GED

.with_column(Column("ucdp_ged_best_sb", from_table = "ged2_cm", from_column = "ged_sb_best_sum_nokgi"))                   
.with_column(Column("ucdp_ged_best_os", from_table = "ged2_cm", from_column = "ged_os_best_sum_nokgi"))   
.with_column(Column("ucdp_ged_best_ns", from_table = "ged2_cm", from_column = "ged_ns_best_sum_nokgi"))          
  
# ACLED
                    
.with_column(Column("acled_sb_fat", from_table = "acled2_cm", from_column = "acled_sb_fat")) 
.with_column(Column("acled_os_fat", from_table = "acled2_cm", from_column = "acled_os_fat")) 
.with_column(Column("acled_ns_fat", from_table = "acled2_cm", from_column = "acled_ns_fat"))
           
.with_column(Column("acled_prx_fat", from_table = "acled2_cm", from_column = "acled_prx_fat"))
.with_column(Column("acled_bat_gov_fat", from_table = "acled2_cm", from_column = "acled_bat_gov_fat"))
.with_column(Column("acled_bat_reb_fat", from_table = "acled2_cm", from_column = "acled_bat_reb_fat"))
                     
.with_column(Column("acled_sb_count", from_table = "acled2_cm", from_column = "acled_sb_count")) 
.with_column(Column("acled_os_count", from_table = "acled2_cm", from_column = "acled_os_count")) 
.with_column(Column("acled_ns_count", from_table = "acled2_cm", from_column = "acled_ns_count"))
  
.with_column(Column("acled_prx_count", from_table = "acled2_cm", from_column = "acled_prx_count")) 
.with_column(Column("acled_pr_count", from_table = "acled2_cm", from_column = "acled_pr_count")) 
                    
       )
      
data_pgm = data_pgm.publish().fetch()

print(f"A dataset with {len(data_pgm.columns)} columns, with "
      f"data between t {min(data_pgm.index.get_level_values(0))} "
      f"and {max(data_pgm.index.get_level_values(0))}. "
      f"({len(np.unique(data_pgm.index.get_level_values(1)))} units)"
     )

 .      o      O      O      o       .      o      O      O      o       .      o      O      O      o       .      o      O      O      o      A dataset with 17 columns, with data between t 1 and 852. (13110 units)


In [13]:
# Subset for given month range

pgm_subset = data_pgm.query(f'month_id >= {StartOfHistory} & month_id <= {EndOfHistory}')
pgm_subset

year  month  country_name  ucdp_ged_best_sb  \
month_id priogrid_gid                                                
121      62356         1990      1  South Africa               0.0   
         79599         1990      1  South Africa               0.0   
         79600         1990      1  South Africa               0.0   
         79601         1990      1  South Africa               0.0   
         80317         1990      1  South Africa               0.0   
...                     ...    ...           ...               ...   
512      190496        2022      8      Bulgaria               0.0   
         190507        2022      8        Turkey               4.0   
         190508        2022      8        Turkey               4.0   
         190510        2022      8        Turkey               4.0   
         190511        2022      8        Turkey               4.0   

                       ucdp_ged_best_os  ucdp_ged_best_ns  acled_sb_fat  \
month_id priogrid_gid                                                     
121      62356                      9.0              41.0           NaN   
         79599                      9.0              41.0           NaN   
         79600                      9.0              41.0           NaN   
         79601                      9.0              41.0           NaN   
         80317                      9.0              41.0           NaN   
...                                 ...               ...           ...   
512      190496                     0.0               0.0           0.0   
         190507                     0.0               0.0          19.0   
         190508                     0.0               0.0          19.0   
         190510                     0.0               0.0          19.0   
         190511                     0.0               0.0          19.0   

                       acled_os_fat  acled_ns_fat  acled_prx_fat  \
month_id priogrid_gid                                              
121      62356                  NaN           NaN            NaN   
         79599                  NaN           NaN            NaN   
         79600                  NaN           NaN            NaN   
         79601                  NaN           NaN            NaN   
         80317                  NaN           NaN            NaN   
...                             ...           ...            ...   
512      190496                 0.0           0.0            0.0   
         190507                 1.0           1.0            0.0   
         190508                 1.0           1.0            0.0   
         190510                 1.0           1.0            0.0   
         190511                 1.0           1.0            0.0   

                       acled_bat_gov_fat  acled_bat_reb_fat  acled_sb_count  \
month_id priogrid_gid                                                         
121      62356                       NaN                NaN             NaN   
         79599                       NaN                NaN             NaN   
         79600                       NaN                NaN             NaN   
         79601                       NaN                NaN             NaN   
         80317                       NaN                NaN             NaN   
...                                  ...                ...             ...   
512      190496                      0.0                0.0             0.0   
         190507                      0.0                0.0            18.0   
         190508                      0.0                0.0            18.0   
         190510                      0.0                0.0            18.0   
         190511                      0.0                0.0            18.0   

                       acled_os_count  acled_ns_count  acled_prx_count  \
month_id priogrid_gid                                                    
121      62356                    NaN             NaN              NaN   
 

#### Download as .csv

In [14]:
pgm_subset.to_csv(desktop+f'API_predictors_pgm_{StartOfHistory}-{EndOfHistory}_fatalities001.csv') 

In [15]:
print("All done")

All done
